<a href="https://colab.research.google.com/github/RuchiChourasia/Deep-learning-with-Python/blob/master/TF-IDF%20with%20SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import All Required Packages**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import csv
import pandas as pd
import numpy as np
# used for encoding
import chardet
# natural language toolkit
import nltk
# wordnet is the database of English language
nltk.download('wordnet')
# stopwords for removing it from review
from nltk.corpus import stopwords
nltk.download('stopwords')
##from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
# 
from nltk.util import ngrams
import string
import math
# importing Linear SVM Classifier 
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#universal variable decleration
dataset=[]
trainData=[]
testData=[]
trainData_label=[]
testData_label=[]


trainDataAll=[]
testDataAll=[]
trainData_labelAll=[]
testData_labelAll=[]


table = str.maketrans({key: None for key in string.punctuation})

label=[]
#data=[]

tfs = {}
tfids = {}
tfidf = {}

tfs_test = {}
tfids_test = {}
tfidf_test = {}

featureDict = {}
featureDict_test = {}
# A global dictionary of features

In [4]:
#df = pd.read_csv('final amazon review datas.csv', encoding='Latin-1')
# r"/content/drive/My Drive/Colab Notebooks/amazon_dataset_final.csv"
df = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/amazon_data.csv", encoding='Latin-1')
df = df.sample(frac=1).reset_index(drop=True)

column_name =df.columns.tolist()
column_name=column_name[:-1]
#print(column_name)
df = df.replace({'LABEL': {'__label1__':'FAKE' , '__label2__':'REAL'}})
label=df['LABEL'].tolist()
#label=label.reshape(-1,1)
df=df.drop('LABEL',axis=1)
display(df.head())
        #display(rawData)
row,col = df.shape
#display(row,col)
#print(len(label))

,DOC_ID,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,16899,5,Y,Wireless,B00V856N3Y,"iPhone 6s Case, TEAM LUXURY [Clarity Series] U...",I absolutely LOVE this case,I absolutely LOVE this case. it arrived very q...
1,17672,2,Y,Lawn and Garden,B008NT2KOE,Lacrosse Color Wireless Automic Weather Statio...,***Long term review***,We bought this for my father for Christmas 201...
2,16054,4,Y,Tools,B001ID4ZY0,SE CC4580 MilitaryLensatic/Prismatic Sighting ...,Great,Classic style compass just like I grew up with...
3,8313,5,N,Video DVD,B00RGQ4674,Outlander: Season One - Volume One,I like it!,I read the books and have enjoyed the series.I...
4,14294,2,Y,Video DVD,B000KBKPIG,Pursuit of Honor - The Rise of George Washington,Nicely done. Very informative.,"We enjoyed this video as a family. However, w..."


In [0]:
def parse(review):
    return (review[7])


In [0]:
def parseAll(review):
  #reviewLine[0], reviewLine[2], reviewLine[3],reviewLine[4], reviewLine[7], label
  return (review[1],review[2],review[3],review[6],review[7])

In [0]:
# TEXT PREPROCESSING AND FEATURE VECTORIZATION
# Input: a string of one review
#t={key: None for key in string.punctuation} #It will return none for puntuations


def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    filtered_tokens=[]
    lemmatized_tokens = []
    stop_words = set(stopwords.words('english'))
    text = text.translate(table) # translation of text is done here punctuations are replaced with none
    for w in text.split(" "):
        if w not in stop_words:
            lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))
            
    
        #print(lemmatized_tokens)
        #filtered_tokens = [' '.join(l) for l in nltk.bigrams(lemmatized_tokens)] + lemmatized_tokens #nltk bigrams will make pair of word occur together mrore frequently
        #here ' '.join(l) means it will join the words from the list (here it is l of iterable) with ' ' delimeter in between. 
    return lemmatized_tokens #lemmatized_tokens  #filtered_tokens
    

In [0]:
def bow(Text):
    #print(Text)
    bowCount = len(Text)
    #display(bowCount)
    return bowCount

def computeTF(localDict, bowCount):
    tfDict = {}
    #bowCount = len(bow)
    for word, count in localDict.items():
        tfDict[word] = count/float(bowCount)
    #display(tfDict)
    return tfDict
    

In [0]:
def computeIDF(docList):
   
    idfDict = {}
    #print(docList)
    N = row ## row
    #display(N)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
      for key, value in doc.items():
        
        if value > 0:
          idfDict[key] += 1
              #print(idfDict)
   
    for word, val in idfDict.items():
      idfDict[word] = math.log10(N / float(val))
   
    return idfDict

In [0]:
def computeTFIDF(tfs, idfs): 
  
  
  for key,val in tfs.items():
    t={}
    for k,v in val.items():
       t[k]=v*idfs[k]
    tfidf[key] = t
      #tfidf[val] = val
  #display(tfidf)
  
  return tfidf
  
def computeTFIDF_test(tfs_test, idfs_t): 
  
  
  for key,val in tfs_test.items():
    t1={}
    for k,v in val.items():
       t1[k]=v*idfs_t[k]
    tfidf_test[key] = t1
      #tfidf[val] = val
  #display(tfidf)
  
  return tfidf_test
  
  

In [0]:


def toFeatureVector(tokens):
    localDict = {}
    for token in tokens:
        if token not in featureDict:
            featureDict[token] = 1
        else:
            featureDict[token] += 1
   
        if token not in localDict:
            localDict[token] = 1
        else:
            localDict[token] += 1
    
    
    return localDict
def toFeatureVector_test(tokens1):
    localDict1 = {}
    for token1 in tokens1:
        if token1 not in featureDict_test:
            featureDict_test[token1] = 1
        else:
            featureDict_test[token1] += 1
   
        if token1 not in localDict1:
            localDict1[token1] = 1
        else:
            localDict1[token1] += 1
    
    
    return localDict1

In [0]:
# parse dataset and extract essential feature from it

## row ##
def tfidf_compute(dataset1):
  length=len(dataset1)
  for i in range(length):
      Text = parse(dataset1[i])
      temp={}
      #data.append((Label,Text))
      ##data.append((Label, computeTF(toFeatureVector(preprocess(Text)),bow(preprocess(Text)))))
      temp={i:computeTF(toFeatureVector(preprocess(Text)),bow(preprocess(Text)))}
      tfs.update(temp)

  idfs = computeIDF([featureDict])
  tfidfs = computeTFIDF(tfs, idfs)
  return tfidfs

def tfidf_compute_test(dataset2):
  length1=len(dataset2)
  for j in range(length1):
      Text1 = parse(dataset2[j])
      temp1={}
      #data.append((Label,Text))
      ##data.append((Label, computeTF(toFeatureVector(preprocess(Text)),bow(preprocess(Text)))))
      temp1={j:computeTF(toFeatureVector_test(preprocess(Text1)),bow(preprocess(Text1)))}
      tfs_test.update(temp1)

  idfs_t = computeIDF([featureDict_test])
  tfidfs_test = computeTFIDF_test(tfs_test, idfs_t)
  return tfidfs_test



In [0]:
def tfidf_computeAll(dataset1):
  length=len(dataset1)
  for i in range(length):
      R,VP,PC,Title,Text = parseAll(dataset1[i])
      temp={}
      #data.append((Label,Text))
      ##data.append((Label, computeTF(toFeatureVector(preprocess(Text)),bow(preprocess(Text)))))
      #temp={i:computeTF(toFeatureVector(preprocess(R+" "+VP+" "+PC+" "+Title+" "+Text)),bow(preprocess(R+" "+VP+" "+PC+" "+Title+" "+Text)))}
      temp={i:computeTF(toFeatureVector(preprocess(f'{R} {VP} {PC} {Title} {Text}')),bow(preprocess(f'{R} {VP} {PC} {Title} {Text}')))}
      
      tfs.update(temp)

  idfs = computeIDF([featureDict])
  tfidfs = computeTFIDF(tfs, idfs)
  return tfidfs

def tfidf_compute_testAll(dataset2):
  length1=len(dataset2)
  for j in range(length1):
      R,VP,PC,Title,Text = parseAll(dataset2[j])
      temp1={}
      #data.append((Label,Text))
      ##data.append((Label, computeTF(toFeatureVector(preprocess(Text)),bow(preprocess(Text)))))
      temp1={j:computeTF(toFeatureVector_test(preprocess(f'{R} {VP} {PC} {Title} {Text}')),bow(preprocess(f'{R} {VP} {PC} {Title} {Text}')))}
      tfs_test.update(temp1)

  idfs_t = computeIDF([featureDict_test])
  tfidfs_test = computeTFIDF_test(tfs_test, idfs_t)
  return tfidfs_test

In [0]:
dataset=list(df.values)


In [0]:

def split(percent):
    dataSamples = len(dataset)
    halfOfData = int(len(dataset)/2)
    trainingSamples = int((percent*dataSamples)/2)
    
  
    x1=dataset[:trainingSamples] + dataset[halfOfData:halfOfData+trainingSamples]
    train_tfidf = tfidf_compute(x1)
  
    for m,n in train_tfidf.items():
      trainData.append((n))
    
   
    for x in label[:trainingSamples]+label[halfOfData:halfOfData+trainingSamples]:
      trainData_label.append(x)
    
    # Test data here
    
    x2=dataset[trainingSamples:halfOfData] + dataset[halfOfData+trainingSamples:]
    test_tfidf=tfidf_compute_test(x2)
    
   
    for m1,n1 in test_tfidf.items():
      testData.append((n1))
    
   
    for y in label[trainingSamples:halfOfData] + label[halfOfData+trainingSamples:]:
      testData_label.append(y)
    
    
  


{'': 0.27013870592086997,
 'able': 0.27013870592086997,
 'clutch': 0.5402774118417399,
 'fine': 0.27013870592086997,
 'fit': 0.27013870592086997,
 'i': 0.27013870592086997,
 'needed': 0.27013870592086997,
 'poulan': 0.27013870592086997,
 'remove': 0.27013870592086997,
 'repair': 0.27013870592086997,
 'this': 0.27013870592086997,
 'tool': 0.5402774118417399,
 'wacker': 0.27013870592086997,
 'weed': 0.27013870592086997}

**Train Data and Test Data**

In [19]:
#print(len(trainData),len(trainData_label))
TRAIN_MAIN_DATA=np.c_[trainData,trainData_label]
TEST_MAIN_DATA=np.c_[testData,testData_label]
TRAIN_MAIN_DATA=TRAIN_MAIN_DATA.tolist()
TEST_MAIN_DATA=TEST_MAIN_DATA.tolist()


19950 19950


**SPLIT Function for All types of Feature Input**

In [0]:
def splitAll(percent):
    dataSamples = len(dataset)
    halfOfData = int(len(dataset)/2)
    trainingSamples = int((percent*dataSamples)/2)
    
    # Train Data here  
  
    x1=dataset[:trainingSamples] + dataset[halfOfData:halfOfData+trainingSamples]
    train_tfidf = tfidf_computeAll(x1)
  
    for m,n in train_tfidf.items():
      trainDataAll.append((n))
    
   
    for x in label[:trainingSamples]+label[halfOfData:halfOfData+trainingSamples]:
      trainData_labelAll.append(x)
    
    # Test Data here
    
    x2=dataset[trainingSamples:halfOfData] + dataset[halfOfData+trainingSamples:]
    test_tfidf=tfidf_compute_testAll(x2)
    
   
    for m1,n1 in test_tfidf.items():
      testDataAll.append((n1))
    
   
    for y in label[trainingSamples:halfOfData] + label[halfOfData+trainingSamples:]:
      testData_labelAll.append(y)
    

In [0]:
splitAll(0.95)

In [0]:
TRAIN_MAIN_DATA_All=np.c_[trainDataAll,trainData_label]
TEST_MAIN_DATA_All=np.c_[testDataAll,testData_label]
TRAIN_MAIN_DATA_All=TRAIN_MAIN_DATA_All.tolist()
TEST_MAIN_DATA_All=TEST_MAIN_DATA_All.tolist()

19950

In [0]:
def trainClassifier(trainclassifier):
    print("Training Classifier")
    pipeline =  Pipeline([('svc', LinearSVC())])
    return SklearnClassifier(pipeline).train(trainclassifier)

In [0]:
def crossValidate(dataset_cv, folds):
    cv_results = []
    foldSize = int(len(dataset_cv)/folds)
    for i in range(0,len(dataset_cv),foldSize):
        classifier = trainClassifier(dataset_cv[:i]+dataset_cv[foldSize+i:])
        y_pred = predictLabels(dataset_cv[i:i+foldSize],classifier)
        a = accuracy_score(list(map(lambda d : d[1], dataset_cv[i:i+foldSize])), y_pred)
        (p,r,f,_) = precision_recall_fscore_support(list(map(lambda d : d[1], dataset_cv[i:i+foldSize])), y_pred, average ='macro')
        #print(a,p,r,f)
        cv_results.append((a,p,r,f))
    cv_results = (np.mean(np.array(cv_results),axis=0))
    return cv_results

In [0]:
def predictLabels(reviewSamples, classifier1):
    return classifier1.classify_many(map(lambda t: t[0], reviewSamples))



In [25]:
print("Mean of cross-validations (Accuracy, Precision, Recall, Fscore): ", crossValidate(TRAIN_MAIN_DATA, 10))

Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Mean of cross-validations (Accuracy, Precision, Recall, Fscore):  [0.67909774 0.67910542 0.67907776 0.67899796]


In [26]:
## Test Accuracy
classifier = trainClassifier(TRAIN_MAIN_DATA)
predictions = predictLabels(TEST_MAIN_DATA, classifier)
true_labels = list(map(lambda d: d[1], TEST_MAIN_DATA))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", r)
print("f1-score: ", f1)

Training Classifier
accuracy:  0.7047619047619048
Precision:  0.7050904287746393
Recall:  0.7054482798732079
f1-score:  0.7046933346881452


In [27]:
print("Mean of cross-validations (Accuracy, Precision, Recall, Fscore): ", crossValidate(TRAIN_MAIN_DATA_All, 10))

Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Training Classifier
Mean of cross-validations (Accuracy, Precision, Recall, Fscore):  [0.82235589 0.82385949 0.82246602 0.82212574]


In [28]:
## Test Accuracy
classifier_All = trainClassifier(TRAIN_MAIN_DATA_All)
predictions_All = predictLabels(TEST_MAIN_DATA_All, classifier_All)
true_labels_All = list(map(lambda d: d[1], TEST_MAIN_DATA_All))
a_All = accuracy_score(true_labels_All, predictions_All)
p_All, r_All, f1_All, _ = precision_recall_fscore_support(true_labels_All, predictions_All, average='macro')
print("accuracy: ", a_All)
print("Precision: ", p_All)
print("Recall: ", r_All)
print("f1-score: ", f1_All)

Training Classifier
accuracy:  0.8219047619047619
Precision:  0.8229029645941499
Recall:  0.8203499869136592
f1-score:  0.820991452991453
